# 1 Data wrangling<a id='2_Data_wrangling'></a>

## 1.1 Contents<a id='Contents'></a>
* [1 Data wrangling](#1_Data_wrangling)
  * [1.1 Contents](#1.1_Contents)
  * [1.2 Introduction](#1.2_Introduction)
  * [1.3 Project Idea and Concept](#1.3_Project_Idea_and_Concept)
  * [1.4 Imports](#1.4_Imports)
  * [1.5 Objectives](#1.5_Objectives)
  * [1.6 Load the data](#1.6_Load_the_data)
  * [1.7 Explore the data](#1.7_Explore_the_data)
  * [1.8 Renaming columns](#1.8_renaming_columns)
  * [1.9 Checking missing values](#1.9_Checking_missing_values)
  * [1.6 Load the data](#1.6_Load_the_data)
  * [1.7 Explore the data](#1.7_Explore_the_data)
  * [1.8 Renaming columns](#1.8_Renaming_columns)
  * [1.9 Checking missing values](#1.9_Checking_missing_values)
  * [1.10 Converting Date columns to Datetime](#1.10_Converting_Date_columns_to_Datetime)
      * [1.10.1 Dropping rows on Date_time subset](#1.10.1_Dropping_rows_on_Date_time_subset)
  * [1.11 Converting Lat & Long columns to Float](#1.11_Converting_Lat_&_Long_columns_to_Float)
  * [1.12 Dropping null rows from Duration Column](#1.12_Dropping_null_rows_from_Duration_Column)
  * [1.13 Getting Country, State, City from Latitude and Longitudes](#1.13_Getting_Country_,_State_,_City_from_Latitude_and_Longitudes)
  * [1.14 Cleaning Duration column to convert all entries to Minutes](#1.14_Cleaning_Duration_column_to_convert_all_entries_to_Minutes)
      * [1.14.1 Function using regex package to get duration](#1.14.1_Function_using_regex_package_to_get_duration)
      * [1.14.2 Applying function to duration column](#1.14.2_Applying_function_to_duration_column)
  * [1.15 Checking and cleaning Shape Column](#1.15_Checking_and_cleaning_Shape_Column)
      * [1.15.1 Checking unique shapes & function to match these in description column](#1.15.1_Checking_unique_shapes_&_function_to_match_these_in_description_column)
      * [1.15.2 Applying function to create shape categories column](#1.15.2_Applying_function_to_create_shape_categories_column)
      * [1.15.3 Checking if shape can be found in description column](#1.15.3_Checking_if_shape_can_be_found_in_description_column)
      * [1.15.4 Dropping null values from final Shape Column](#1.15.4_Dropping_null_values_from_final_Shape_Column)
  * [1.16 Getting Year and Month From Spotted datetime](#1.16_Getting_Year_and_Month_From_Spotted_datetime)
  * [1.17 Saving File](#Saving_File)
  

## 1.2 Introduction<a id='1.2_Introduction'></a>

This Notebook features the second step in the Data Science Method Workflow after problem identification which is data Wrangling. It comprises of collecting, defining and cleaning data for furthur exploratory analysis so as to have a clean uniform data with proper data types

## 1.3 Project Idea and Concept<a id='1.3_Project_Idea_and_Concept'></a>

Idea behind this project is to analyze and mine the documented data available to us based on UFO sightings and find interesting trends and predict some behavior in terms of their shape, size, color, etc. Most of the analysis is based on visualizing maps to understand the geographies of UFO sighting. The basic variables or attributes we have in this data are date sighted, length, place of sighting, shape and a description from the observer.

An unidentified flying object (UFO) is any aerial phenomenon that cannot immediately be identified or explained. Most UFOs are identified on investigation as conventional objects or phenomena. The term is widely used for claimed observations of extraterrestrial spacecraft. The data has been collected from NUFORC.


## 1.4 Imports<a id='1.4_Imports'></a>

In [7]:
## ---- Importing the required libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import pycountry
import re
import reverse_geocoder as rg


## 1.5 Objectives<a id='1.5_Objectives'></a>

Main objectives in thiss workbook:
* To address the data types abd change them as needed
* To clean the columns
* To identify missing values
* To identify the target variable for modelling

## 1.6 Load the data<a id='1.6_Load_the_data'></a>

In [8]:
## Importing the CSV file

ufo_data = pd.read_csv("UFO.csv", low_memory = False, na_values = ['UNKNOWN','UNK'], na_filter = True, skip_blank_lines = True)

## 1.7 Explore the data<a id='1.7_Explore_the_data'></a>

In [9]:
# Shape of Data
ufo_data.shape

(88824, 11)

In [10]:
ufo_data.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
dtype: object

In [11]:
ufo_data.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude'],
      dtype='object')

In [12]:
ufo_data.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [13]:
ufo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88824 entries, 0 to 88823
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              88824 non-null  object 
 1   city                  88679 non-null  object 
 2   state                 81352 non-null  object 
 3   country               76314 non-null  object 
 4   shape                 85865 non-null  object 
 5   duration (seconds)    88822 non-null  object 
 6   duration (hours/min)  85755 non-null  object 
 7   comments              88789 non-null  object 
 8   date posted           88824 non-null  object 
 9   latitude              88824 non-null  object 
 10  longitude             88824 non-null  float64
dtypes: float64(1), object(10)
memory usage: 7.5+ MB


In [14]:
ufo_1 = ufo_data.copy()

In [15]:
# Dropping some columns
ufo_1 = ufo_1.drop(['duration (seconds)','country', 'state', 'city'], axis = 1)

In [16]:
ufo_1.columns

Index(['datetime', 'shape', 'duration (hours/min)', 'comments', 'date posted',
       'latitude', 'longitude'],
      dtype='object')

## 1.8 Renaming columns<a id='1.8_Renaming_columns'></a>

In [17]:
## Renaming some columns
ufo_1 = ufo_1.rename({'datetime':"Date_time", "shape":"Shape", "duration (hours/min)":"Duration_minutes", "comments":"Description","date posted" : "Date_posted", "latitude":"Lat", "longitude":"Long"}, axis = 1)

In [18]:
ufo_1.columns

Index(['Date_time', 'Shape', 'Duration_minutes', 'Description', 'Date_posted',
       'Lat', 'Long'],
      dtype='object')

## 1.9 Checking missing values<a id='1.9_Checking_missing_values'></a>

In [19]:
# checking Null values

ufo_1.isnull().sum()

Date_time              0
Shape               2959
Duration_minutes    3069
Description           35
Date_posted            0
Lat                    0
Long                   0
dtype: int64

Shape & Duration Columns each have 2959 and 3069 null values respectively

### Checking data types

In [20]:
ufo_1.dtypes

Date_time            object
Shape                object
Duration_minutes     object
Description          object
Date_posted          object
Lat                  object
Long                float64
dtype: object

## 1.10 Converting Date columns to Datetime<a id='1.10_Converting_Date_columns_to_Datetime'></a>

In [21]:
 ufo_1['Date_time'] = ufo_1['Date_time'].map({t:pd.to_datetime(t,errors="coerce") for t in ufo_1.Date_time.unique()})

In [22]:
 ufo_1['Date_posted'] = ufo_1['Date_posted'].map({t:pd.to_datetime(t,errors="coerce") for t in ufo_1.Date_posted.unique()})

In [23]:
ufo_1.dtypes

Date_time           datetime64[ns]
Shape                       object
Duration_minutes            object
Description                 object
Date_posted         datetime64[ns]
Lat                         object
Long                       float64
dtype: object

After converting these columns to datetime64 type , there were some entries in Date_time column which were 5 digit entries,
after connverting them to proper datetime using strftime(), they tyrned out ti be some dates in the futures so next few 
steps are to drop those rows so, I have subset it on Date_tiome column in dropna command

### 1.10.1 Dropping rows on Date_time subset<a id='1.10.1_Dropping_rows_on_Date_time_subset'></a>

In [24]:
ufo_1 = ufo_1.dropna(axis = 0, subset = ['Date_time'])

In [25]:
ufo_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87595 entries, 0 to 88823
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date_time         87595 non-null  datetime64[ns]
 1   Shape             84904 non-null  object        
 2   Duration_minutes  84913 non-null  object        
 3   Description       87565 non-null  object        
 4   Date_posted       87595 non-null  datetime64[ns]
 5   Lat               87595 non-null  object        
 6   Long              87595 non-null  float64       
dtypes: datetime64[ns](2), float64(1), object(4)
memory usage: 5.3+ MB


In [26]:
ufo_a = ufo_1.copy()

In [27]:
ufo_a.reset_index(drop = True)

,Date_time,Shape,Duration_minutes,Description,Date_posted,Lat,Long
0,1949-10-10 20:30:00,cylinder,45 minutes,This event took place in early fall around 194...,2004-04-27,29.8830556,-97.941111
1,1949-10-10 21:00:00,light,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.38421,-98.581082
2,1955-10-10 17:00:00,circle,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.2,-2.916667
3,1956-10-10 21:00:00,circle,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.9783333,-96.645833
4,1960-10-10 20:00:00,light,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.4180556,-157.803611
...,...,...,...,...,...,...,...
87590,2013-09-09 22:00:00,other,hour,Napa UFO&#44,2013-09-30,38.2972222,-122.284444
87591,2013-09-09 22:20:00,circle,5 seconds,Saw a five gold lit cicular craft moving fastl...,2013-09-30,38.9011111,-77.265556
87592,2013-09-09 23:00:00,cigar,17 minutes,2 witnesses 2 miles apart&#44 Red &amp; White...,2013-09-30,35.6527778,-97.477778
87593,2013-09-09 23:00:00,diamond,2 nights,On September ninth my wife and i noticed stran...,2013-09-30,34.3769444,-82.695833


## 1.11 Converting Lat & Long columns to Float<a id='1.11_Converting_Lat_&_Long_columns_to_Float'></a>

In [28]:
ufo_a.iloc[47931,5] = 33.200088

In [29]:
## Changing Lat column to Float Type

ufo_a['Lat'] = ufo_a['Lat'].astype(float)

In [30]:
ufo_a.dtypes

Date_time           datetime64[ns]
Shape                       object
Duration_minutes            object
Description                 object
Date_posted         datetime64[ns]
Lat                        float64
Long                       float64
dtype: object

In [32]:
ufo_a.sample(5)

,Date_time,Shape,Duration_minutes,Description,Date_posted,Lat,Long
34901,1966-03-15 14:00:00,circle,20sec.,we were driving down the road i look up to the...,2001-08-20,39.646111,-75.871944
1570,2012-10-13 22:00:00,circle,3 minutes,12 Redish flying objects over Isle of Palms,2012-10-30,32.786667,-79.795000
55646,2004-06-19 18:30:00,light,10 minutes,Los Angeles my boyfriend and I had stopped at ...,2004-06-23,34.052222,-118.242778
77415,2001-08-27 05:30:00,circle,10min+,Extremely bright light circle with vapor relea...,2001-10-12,64.837778,-147.716389
39177,2003-03-08 13:00:00,other,1-10seconds,I was at the st. louis zoo about 50ft away fro...,2003-03-21,38.627222,-90.197778


In [33]:
ufo_a.columns

Index(['Date_time', 'Shape', 'Duration_minutes', 'Description', 'Date_posted',
       'Lat', 'Long'],
      dtype='object')

## 1.12 Dropping null rows from Duration Column<a id='1.12_Dropping_null_rows_from_Duration_Column'></a>

In [34]:
# Dropping null values from Duartion Column
ufo_b = ufo_a.dropna(subset = ['Duration_minutes'])

In [35]:
ufo_b = ufo_b.reset_index(drop = True)

In [36]:
ufo_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84913 entries, 0 to 84912
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date_time         84913 non-null  datetime64[ns]
 1   Shape             82815 non-null  object        
 2   Duration_minutes  84913 non-null  object        
 3   Description       84898 non-null  object        
 4   Date_posted       84913 non-null  datetime64[ns]
 5   Lat               84913 non-null  float64       
 6   Long              84913 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(3)
memory usage: 4.5+ MB


<b> After dropping null from datetime and duration columns we have 84913 rows </b>

## 1.13 Getting Country, State, City from Latitude and Longitudes<a id='1.13_Getting_Country_,_State_,_City_from_Latitude_and_Longitudes'></a>

In [37]:
ufo_b['lat_long'] = list(zip(ufo_b.Lat , ufo_b.Long))

In [38]:
ufo_b.head()

,Date_time,Shape,Duration_minutes,Description,Date_posted,Lat,Long,lat_long
0,1949-10-10 20:30:00,cylinder,45 minutes,This event took place in early fall around 194...,2004-04-27,29.883056,-97.941111,"(29.8830556, -97.9411111)"
1,1949-10-10 21:00:00,light,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.384210,-98.581082,"(29.38421, -98.581082)"
2,1955-10-10 17:00:00,circle,20 seconds,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.200000,-2.916667,"(53.2, -2.916667)"
3,1956-10-10 21:00:00,circle,1/2 hour,My older brother and twin sister were leaving ...,2004-01-17,28.978333,-96.645833,"(28.9783333, -96.6458333)"
4,1960-10-10 20:00:00,light,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.418056,-157.803611,"(21.4180556, -157.8036111)"


<b> Using reverse geocoder (rg) package </b> 

In [39]:
location = rg.search(list(ufo_b['lat_long']))

Loading formatted geocoded file...


In [40]:
ufo_b['Country'] = [p["cc"] for p in location]

In [41]:
ufo_b['State'] = [p["admin1"] for p in location]

In [42]:
ufo_b['City'] = [p["name"] for p in location]

In [43]:
ufo_b = ufo_b.drop(['Lat','Long'], axis = 1)

In [44]:
ufo_b.sample(10)

,Date_time,Shape,Duration_minutes,Description,Date_posted,lat_long,Country,State,City
15257,2004-11-04 15:45:00,other,10 minutes,I was feeding the cow&#39s when suddenly 4 uf...,2005-02-22,"(-34.603723, -58.381593)",AR,Buenos Aires F.D.,Buenos Aires
49784,1967-06-01 22:00:00,disk,3or4 minutes,1967 object seen in gerry&#44 new york made no...,2004-04-27,"(42.1933333, -79.2488889)",US,New York,Falconer
70758,1967-08-16 16:30:00,changing,5-10 minutes,Dark &quot;disc&quot; over Newton Creek in Cam...,2005-12-16,"(39.9258333, -75.12)",US,New Jersey,Camden
31022,2000-02-04 21:30:00,circle,3 min.,Yellow-orange objects moving NNW and N &#39til...,2000-02-16,"(34.61, -112.315)",US,Arizona,Prescott Valley
62131,2009-07-21 23:00:00,light,30 seconds,Mysterious Bright Light moving very fast acros...,2009-08-05,"(39.5297222, -119.8127778)",US,Nevada,Reno
21188,2002-12-23 03:30:00,teardrop,five minutes,teardrop/pear&#44 black/blue&#44 wings(?)&#44 ...,2003-03-21,"(38.8402778, -77.4291667)",US,Virginia,Centreville
30266,2005-02-25 22:00:00,circle,15 min.,The cirular craft went straight up and disappe...,2005-04-16,"(39.6411111, -85.1411111)",US,Indiana,Connersville
15135,2013-11-03 20:10:00,chevron,7 seconds,Glowing gold specks in an arch shape moving qu...,2013-11-11,"(39.6133333, -105.0161111)",US,Colorado,Littleton
67818,1958-08-10 21:30:00,light,20 minutes,At dusk&#44 a light appeared out of the SE sk...,2012-11-19,"(47.6105556, -122.1994444)",US,Washington,Bellevue
55490,2012-06-28 21:04:00,fireball,10 seconds,Orange ball of fire seen by family in car,2012-07-04,"(40.26, -74.2741667)",US,New Jersey,Freehold


In [45]:
ufo_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84913 entries, 0 to 84912
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date_time         84913 non-null  datetime64[ns]
 1   Shape             82815 non-null  object        
 2   Duration_minutes  84913 non-null  object        
 3   Description       84898 non-null  object        
 4   Date_posted       84913 non-null  datetime64[ns]
 5   lat_long          84913 non-null  object        
 6   Country           84913 non-null  object        
 7   State             84913 non-null  object        
 8   City              84913 non-null  object        
dtypes: datetime64[ns](2), object(7)
memory usage: 5.8+ MB


## 1.14 Cleaning Duration column to convert all entries to Minutes<a id='1.14_Cleaning_Duration_column_to_convert_all_entries_to_Minutes'></a>

### 1.14.1 Function using regex package to get duration<a id='1.14.1_Function_using_regex_package_to_get_duration'></a>

In [46]:
# Using regex package as re, trying to look for some patterns and converting it all into minutes
# First Function is to check for type digit followed by unit: (/d+) (/w+) 

#--------------------------------------------------------------------

### 1. Check for certain string and replace it my numerical unit, such as few by 3.5, several by 7.5, couple by 2, 1/2 by 0.5
### 2. Look for pattern as digit.digit or digit
###3. look for units in terms of seconds, minutes, hours

#------------------------------------------------------------------------
# Func takes string as the input and returns string after multiplying accordingly to the unit

#--------------------------------------------------------------------

def duration_clean(string):
    string = string.replace("few", "3.5").replace("1/2","0.5").replace("several","7.5").replace("couple","2").replace("?","").replace("one","1").replace("two","2").replace("five","5").replace("ten","10").replace("three","3").replace('a',"1")
    #aplhanumeric = ["(\d+)(\+w)"]
    numeric = ["(\d+\.\d+)","(\d+)"]
    unit = ["se[cs]", "secon[ds]","mi[ns]", "mi[mn]ut[es]","h[rs]", "hou[rs]"]
    unitConversion = {'s':1/60, 'm':1, 'h':60, '6':60}
    try:
        m_1 = re.search(re.compile("|".join(str(x) for x in numeric)), string).group(0)
        m_2 = re.search(re.compile("|".join(str(x) for x in unit)), string.lower()).group(0)[0]
        string = float(m_1)*unitConversion[m_2]
    except:
        s = np.nan
    return string

### 1.14.2 Applying function to duration column<a id='1.14.2_Applying_function_to_duration_column'></a>

In [47]:
ufo_b['Duration_minutes'] = ufo_b['Duration_minutes'].apply(duration_clean)

In [49]:
ufo_b.sample(7)

,Date_time,Shape,Duration_minutes,Description,Date_posted,lat_long,Country,State,City
33001,2004-03-13 20:00:00,light,23:00,Light hovered over several towns &#44 it didn&...,2004-03-17,"(41.7383333, -95.7025)",US,Iowa,Woodbine
29748,2005-02-22 18:00:00,light,0.166667,Bright orange light low in night sky,2005-02-24,"(33.145405, -96.839054)",US,Texas,Frisco
10989,2001-11-18 02:22:00,other,0.333333,Slow-flying crescent made up of lots of little...,2001-11-20,"(47.6063889, -122.3308333)",US,Washington,Seattle
16213,2005-01-16 04:00:00,cylinder,current,Rainbow light in sky above Phoenix,2005-01-19,"(33.4483333, -112.0733333)",US,Arizona,Phoenix
60318,2001-07-15 00:29:00,circle,2.0,25 lights&#44 75 witnesses over cataret Nj on ...,2001-08-05,"(40.5772222, -74.2286111)",US,New Jersey,Carteret
27837,2003-02-13 14:00:00,other,10.0,Floating bell shaped object with pure white li...,2003-03-21,"(34.6391667, -87.1916667)",US,Alabama,Trinity
80082,2003-09-19 14:30:00,teardrop,240.0,Glowing tear drop-shape in the South West,2004-01-17,"(35.61251, -106.350511)",US,New Mexico,Santo Domingo Pueblo


In [50]:
ufo_b[ufo_b['Duration_minutes'].apply(lambda x: isinstance(x, str))].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6729 entries, 93 to 84912
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date_time         6729 non-null   datetime64[ns]
 1   Shape             6518 non-null   object        
 2   Duration_minutes  6729 non-null   object        
 3   Description       6727 non-null   object        
 4   Date_posted       6729 non-null   datetime64[ns]
 5   lat_long          6729 non-null   object        
 6   Country           6729 non-null   object        
 7   State             6729 non-null   object        
 8   City              6729 non-null   object        
dtypes: datetime64[ns](2), object(7)
memory usage: 525.7+ KB


<b> Out of 84913 rows, In duration column, 6729 rows are of string type where no numerical value is present 
so its hard to detect duration in these rows whereas rest are float type as reuired for out clean dataset. 
I have kept these rows for now. Ee will deal with them in our next step i.e EDA </b>

In [51]:
ufo_b.isnull().sum()

Date_time              0
Shape               2098
Duration_minutes       0
Description           15
Date_posted            0
lat_long               0
Country                0
State                  0
City                   0
dtype: int64

In [52]:
ufo_c = ufo_b.dropna(subset = ['Description'])

## 1.15 Checking and cleaning Shape Column<a id='1.15_Checking_and_cleaning_Shape_Column'></a>

In [53]:
ufo_c.isnull().sum()

Date_time              0
Shape               2091
Duration_minutes       0
Description            0
Date_posted            0
lat_long               0
Country                0
State                  0
City                   0
dtype: int64

<b> There are 2091 null values in shape column but some shapes are also mentioned in Description column so, below mentioned steps are to check if for any null value in shape column, can we get shape from the description for that record. I have writtena function to check for the shape keywords in description column and the match them with shape column to get the new shape column </b>

In [54]:
ufo_c['Shape'].unique()

array(['cylinder', 'light', 'circle', 'sphere', 'disk', 'fireball',
       'unknown', 'oval', 'other', 'cigar', 'rectangle', 'chevron',
       'triangle', 'formation', nan, 'delta', 'changing', 'egg', 'flash',
       'diamond', 'cross', 'teardrop', 'cone', 'pyramid', 'round',
       'crescent', 'flare', 'hexagon', 'changed'], dtype=object)

In [55]:
ufo_c['Shape'].value_counts()

light        17420
triangle      8227
circle        8073
fireball      6395
unknown       6040
other         6005
disk          5647
sphere        5621
oval          3959
formation     2604
cigar         2158
changing      2071
flash         1425
rectangle     1368
cylinder      1333
diamond       1252
chevron        990
egg            818
teardrop       787
cone           348
cross          251
delta            7
round            2
crescent         2
hexagon          1
changed          1
pyramid          1
flare            1
Name: Shape, dtype: int64

### 1.15.1 Checking unique shapes & function to match these in description column<a id='1.15.1_Checking_unique_shapes_&_function_to_match_these_in_description_column'></a>

In [56]:
# Creating a list of all uniques shapes mentioned

shapes = [i.lower() for i in ufo_c['Shape'].value_counts().index if i not in ['unknown','other']]

In [57]:
print(shapes)

['light', 'triangle', 'circle', 'fireball', 'disk', 'sphere', 'oval', 'formation', 'cigar', 'changing', 'flash', 'rectangle', 'cylinder', 'diamond', 'chevron', 'egg', 'teardrop', 'cone', 'cross', 'delta', 'round', 'crescent', 'hexagon', 'changed', 'pyramid', 'flare']


In [58]:
## Function to chech if shape column words exist in description column

#-----------------------------------------------------------------------

# Takes in the column and shape list and returns a list of shapes

def shape(r, shape_list):
    Desc = r.lower().split()
    shape_count = dict(zip(list(shape_list), [0] * len(shape_list)))
    for word in Desc:
        if word in shape_list:
            shape_count[word] += 1
    shape_count = {k[0].upper()+k[1:]:v for k, v in shape_count.items() if v}
    return list(shape_count.keys())


### 1.15.2 Applying function to create shape categories column<a id='1.15.2_Applying_function_to_create_shape_categories_column'></a>

In [59]:
ufo_c['Shape Categories'] = ufo_c['Description'].apply(lambda x: shape(x, shapes))

D:\Anacon\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
ufo_c.head()

,Date_time,Shape,Duration_minutes,Description,Date_posted,lat_long,Country,State,City,Shape Categories
0,1949-10-10 20:30:00,cylinder,45.0,This event took place in early fall around 194...,2004-04-27,"(29.8830556, -97.9411111)",US,Texas,San Marcos,[]
1,1949-10-10 21:00:00,light,60.0,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,"(29.38421, -98.581082)",US,Texas,Lackland Air Force Base,[]
2,1955-10-10 17:00:00,circle,0.333333,Green/Orange circular disc over Chester&#44 En...,2008-01-21,"(53.2, -2.916667)",GB,England,Blacon,[]
3,1956-10-10 21:00:00,circle,30.0,My older brother and twin sister were leaving ...,2004-01-17,"(28.9783333, -96.6458333)",US,Texas,Edna,[]
4,1960-10-10 20:00:00,light,15.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,"(21.4180556, -157.8036111)",US,Hawaii,Kane'ohe,[]


### 1.15.3 Checking if shape can be found in description column<a id='1.15.3_Checking_if_shape_can_be_found_in_description_column'></a>

In [61]:
## Checking if for null vlaues in Shaoe column does the new column ( Shape categories has any value) 
## and assigning it into a shape_final column

shape = []
for x, y in zip(ufo_c['Shape'], ufo_c['Shape Categories']):
    if (pd.isnull(x)) and (y != []):
        shape.append(y)
    elif (pd.isnull(x)) and (y == []):
        shape.append(x)
    elif (x in ['unknown','other']) and (y != []):
        shape.append(y)
    elif (x in ['unknown','other']) and (y == []):
        shape.append(y + [x])
    elif (x not in ['Unknown','Other']) and (x not in y):
        shape.append(y + [x])
    else:
        shape.append(y)
ufo_c['Shape_final'] = shape

D:\Anacon\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [62]:
ufo_c.head()

,Date_time,Shape,Duration_minutes,Description,Date_posted,lat_long,Country,State,City,Shape Categories,Shape_final
0,1949-10-10 20:30:00,cylinder,45.0,This event took place in early fall around 194...,2004-04-27,"(29.8830556, -97.9411111)",US,Texas,San Marcos,[],[cylinder]
1,1949-10-10 21:00:00,light,60.0,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,"(29.38421, -98.581082)",US,Texas,Lackland Air Force Base,[],[light]
2,1955-10-10 17:00:00,circle,0.333333,Green/Orange circular disc over Chester&#44 En...,2008-01-21,"(53.2, -2.916667)",GB,England,Blacon,[],[circle]
3,1956-10-10 21:00:00,circle,30.0,My older brother and twin sister were leaving ...,2004-01-17,"(28.9783333, -96.6458333)",US,Texas,Edna,[],[circle]
4,1960-10-10 20:00:00,light,15.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,"(21.4180556, -157.8036111)",US,Hawaii,Kane'ohe,[],[light]


<b> There are around 1500 null values in Shape Final column so after dropping thise rows final count of rows is 83392 </b>

### 1.15.4 Dropping null values from final Shape Column<a id='1.15.4_Dropping_null_values_from_final Shape_Column'></a>

In [63]:
# Droping null rows form shape_final column

ufo_d = ufo_c.dropna(subset = ['Shape_final'])

In [64]:
ufo_d = ufo_d.reset_index(drop = True)

In [65]:
# Dropping Shape and Shape categories columns

ufo_d.drop(['Shape', 'Shape Categories'], axis = 1, inplace = True)

In [66]:
ufo_d.columns

Index(['Date_time', 'Duration_minutes', 'Description', 'Date_posted',
       'lat_long', 'Country', 'State', 'City', 'Shape_final'],
      dtype='object')

In [67]:
ufo_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83392 entries, 0 to 83391
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date_time         83392 non-null  datetime64[ns]
 1   Duration_minutes  83392 non-null  object        
 2   Description       83392 non-null  object        
 3   Date_posted       83392 non-null  datetime64[ns]
 4   lat_long          83392 non-null  object        
 5   Country           83392 non-null  object        
 6   State             83392 non-null  object        
 7   City              83392 non-null  object        
 8   Shape_final       83392 non-null  object        
dtypes: datetime64[ns](2), object(7)
memory usage: 5.7+ MB


<b> Now We have equal number of rows for all columns </b>

In [69]:
ufo_d.sample(5)

,Date_time,Duration_minutes,Description,Date_posted,lat_long,Country,State,City,Shape_final
69920,2012-08-17 20:00:00,5.0,2 orange/red orbs hovering and moving erratica...,2012-09-24,"(40.256213, -75.463788)",US,Pennsylvania,Schwenksville,[other]
75408,2004-08-09 21:00:00,0.033333,object moving at a high rate of speed.,2005-09-02,"(33.8358333, -79.0480556)",US,South Carolina,Conway,[unknown]
36573,2007-03-07 19:30:00,5.0,Huge musquito like golden object over Ionian S...,2007-04-27,"(0.0, 0.0)",GH,Western,Takoradi,[other]
15524,2005-01-15 19:30:00,210.0,Multiple types of objects in night sky of cent...,2005-01-19,"(29.1869444, -82.1402778)",US,Florida,Ocala,[triangle]
34458,2003-03-23 02:30:00,very f1st,HBCCUFO CANADIAN REPORT: I was wondering if ...,2003-04-22,"(54.5, -128.583333)",CA,British Columbia,Terrace,[light]


## 1.16 Getting Year and Month From Spotted datetime<a id='1.16_Getting_Year_and_Month_From_Spotted_datetime'></a>

In [70]:
ufo_d['Year'] = ufo_d['Date_time'].dt.year

In [71]:
ufo_d.head()

,Date_time,Duration_minutes,Description,Date_posted,lat_long,Country,State,City,Shape_final,Year
0,1949-10-10 20:30:00,45.0,This event took place in early fall around 194...,2004-04-27,"(29.8830556, -97.9411111)",US,Texas,San Marcos,[cylinder],1949
1,1949-10-10 21:00:00,60.0,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,"(29.38421, -98.581082)",US,Texas,Lackland Air Force Base,[light],1949
2,1955-10-10 17:00:00,0.333333,Green/Orange circular disc over Chester&#44 En...,2008-01-21,"(53.2, -2.916667)",GB,England,Blacon,[circle],1955
3,1956-10-10 21:00:00,30.0,My older brother and twin sister were leaving ...,2004-01-17,"(28.9783333, -96.6458333)",US,Texas,Edna,[circle],1956
4,1960-10-10 20:00:00,15.0,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,"(21.4180556, -157.8036111)",US,Hawaii,Kane'ohe,[light],1960


In [72]:
ufo_d['Month'] = ufo_d['Date_time'].dt.month_name()

In [73]:
ufo_d.sample(10)

,Date_time,Duration_minutes,Description,Date_posted,lat_long,Country,State,City,Shape_final,Year,Month
33541,2012-03-18 22:00:00,2.0,Amber colored silent spherical fireball rising...,2012-05-13,"(30.5166667, -86.4822222)",US,Florida,Niceville,"[Fireball, fireball]",2012,March
24855,2006-12-09 20:01:00,1.0,Bright red &quot;fireball&quot; lasting about ...,2006-12-14,"(30.1125, -85.2005556)",US,Florida,Wewahitchka,[fireball],2006,December
32055,1997-03-12 18:20:00,10.0,Two objects doing imposible manovers,2004-12-14,"(54.911944, -1.383333)",GB,England,Sunderland,[sphere],1997,March
40740,2011-04-26 23:10:00,5.0,San Jose - red object in sky heading south.,2011-05-02,"(37.3394444, -121.8938889)",US,California,San Jose,[light],2011,April
14995,2004-11-04 01:15:00,120.0,Very unusual &quot;star&quot; to the SE Seattl...,2004-11-09,"(47.6063889, -122.3308333)",US,Washington,Seattle,[light],2004,November
75701,2007-09-10 05:00:00,10.0,Stationary red&#44 blue&#44 green&#44 white li...,2007-10-08,"(30.6277778, -96.3341667)",US,Texas,College Station,[Light],2007,September
2979,2013-10-18 20:00:00,6.0,White/blue sphere maneuvering around commerica...,2013-10-23,"(39.74844, -84.283398)",US,Ohio,Drexel,"[Sphere, sphere]",2013,October
35627,2013-03-30 21:18:00,3.0,3 Possible orange UFO spheres over Phoenix Ari...,2013-05-15,"(33.4483333, -112.0733333)",US,Arizona,Phoenix,[fireball],2013,March
51879,2006-06-18 01:10:00,0.033333,Family of four sees bright flash in night sky.,2006-07-16,"(38.1916667, -120.8280556)",US,California,Valley Springs,"[Flash, flash]",2006,June
24033,2012-12-06 18:47:00,0.166667,Five glowing fiery lights seen looking north f...,2012-12-20,"(33.2147222, -97.1327778)",US,Texas,Denton,[light],2012,December


<b> Our Final cleaned data set has 11 columns and 83392 records </b>

## 1.17 Saving File<a id='Saving_File'></a>

In [80]:
ufo_d.to_csv("ufo_cleaned.csv", header = True, index = False)

</b> Credits to https://github.com/Dascienz/ufo-sightings/blob/master </b>